Integrantes: 

* Daniel Julián Rojas Cruz
* Johan Tanta Villanueva

# Redes Bayesianas - Inteligencia Artificial


Se presenta la implementación de la función count que retorna el número de filas que satisfacen las igualdades


In [2]:
import pandas as pd
import numpy as np


df = pd.DataFrame(np.random.randint(0,10,size=(10, 3)), columns=list('ABC'))


def count(dataframe, index_list, values_list):
    resultado = dataframe
    for i in range(len(index_list)):
        index = index_list[i]
        value = values_list[i]

    #filtrar la columna 
        resultado = resultado[resultado[index] == value]
    c =resultado.shape[0]
    return c



## Objetivo 1
El objetivo 1 consiste en estimar la marginal para una variable en consulta a partir de un dataset

### Implementación del Objetivo 1


In [3]:

# Esta función recive el nombre del archivo CSV y devuelve un dataframe de Pandas
def leer_dataset(nom_archivo):
    return pd.read_csv(nom_archivo)


def cardinalidad(df, X):
    return df[X].unique()


# Esta funcióm devuelve un reporte del factor que contiene a la variable X
def estimar_marginal(df,X, alpha):
    card = cardinalidad(df,X) # lista de valores únicos
    
    nfilas = df.shape[0]
    
    
    probList = []
    for valor in card:
        m = count(df,[X],[valor])
        p = (m + alpha)/(nfilas + len(card)*alpha)
        probList.append(p)
    probColName = "P"
    columns = {X:card, probColName: probList}
    
    
    return pd.DataFrame(columns)



df = leer_dataset("weather - weather.csv")
marginal = estimar_marginal(df,'play',1)
marginal

,play,P
0,yes,0.625
1,no,0.375


## Objetivo 2
El objetivo 2 consiste en estimar la condicional para un par o más variables en consulta a partir del dataset previo

### Implementación del Objetivo 2


In [4]:


def estimar_condicional(df, X, Y, alpha):
    # hallar el número total de filas 
    nFilasResult = 1
    nfilas = df.shape[0]
    cardTarget = cardinalidad(df,X)
    nFilasResult = nFilasResult*len(cardTarget)
    
    
    cardY = {}
    for var in Y:
        cardY[var] = cardinalidad(df,var)
        nFilasResult = nFilasResult*len(cardY[var])
    
    
    
    columns = {}
    
    n_rep = 1
    
    colTarget = [cardTarget[i%len(cardTarget)] for i in range(nFilasResult)]
    
    columns[X] = colTarget
    n_rep = n_rep * len(cardTarget)
    
    for var, card in cardY.items():
        columns[var] = [ card[int(i/n_rep)%len(card)] for i in range(nFilasResult) ]
        
        n_rep = n_rep * len(card)

    
    result = pd.DataFrame(columns)
    probList = []
    
    prodCardY = 1
    for var in Y:
        prodCardY = prodCardY* len(cardY[var])
        
    
    #Iteramos por cada fila del dataframe y hallamos la probabilidad condicional
    for i in result.index:
        valTarget = result[X][i]
    
        
        valPadres = []
        for var in Y:
            valPadres.append(result[var][i])
      
        totalVar = [i for i in Y]
        totalVar.append(X)
        totalVal = valPadres
        totalVal.append(valTarget)
        
        
        prob_X_Y = (count(df,totalVar,totalVal) + alpha) / (nfilas + prodCardY*len(cardTarget)*alpha) 
        
        prob_Y = (count(df,Y,valPadres) + alpha) / (nfilas + prodCardY*alpha)
        p = prob_X_Y/prob_Y
        probList.append(p)
    # normalizamos las probabilidades
    
    for i in range(int (nFilasResult / len(cardTarget))):
        suma = 0
        for j in range(len(cardTarget)):
            suma = suma + probList[i*len(cardTarget) + j]
        
        for j in range(len(cardTarget)):
            probList[i*len(cardTarget) + j] = probList[i*len(cardTarget) + j] / suma 
    
    result['P'] = probList
    return result

cond = estimar_condicional(df,'outlook', ['windy','play'],1)
cond

,outlook,windy,play,P
0,overcast,False,yes,0.333333
1,rainy,False,yes,0.444444
2,sunny,False,yes,0.222222
3,overcast,True,yes,0.500000
4,rainy,True,yes,0.166667
5,sunny,True,yes,0.333333
6,overcast,False,no,0.200000
7,rainy,False,no,0.200000
8,sunny,False,no,0.600000
9,overcast,True,no,0.166667


## Inferencia


### Implementación de la función inferencia


In [5]:


def estimar_marginal_valor(df,X, valor, alpha):
    card = cardinalidad(df,X) # lista de valores únicos
    
    nfilas = df.shape[0]
    
    
    m = count(df,[X],[valor])
    p = (m + alpha)/(nfilas + len(card)*alpha)
    
    return p

print(estimar_marginal_valor(df, 'play', 'no', 1))



def estimar_condicional_valor(df, X, Y, list_values, alpha):
    # hallar el número total de filas 
    df_cond = estimar_condicional(df,X,Y, alpha)

    
    resultado = df_cond
  
    for i in range(len(Y)):
        index = Y[i]
        value = list_values[i]

        #filtrar la columna 
        resultado = resultado[resultado[index] == value]
    
    
    return resultado






def get_parents(grafo):
    
    result = {}
    
    for node, list_adyacentes in grafo.items():
        if node not in result:
            result[node] = []
        
        if (len(list_adyacentes) > 0):  
            for child in list_adyacentes:
                if child not in result:
                    result[child] = [node]
                else:
                    result[child].append(node)
                
    
    return result


class Red_Bayesiana:
    def __init__(self, df, grafo, alpha):
        self.alpha = alpha
        self.estructura = grafo
        self.parents = get_parents(self.estructura)
        self.factores = self.generate(df)
        self.card = self.get_cards(df)
    
    def get_cards(self, df):
        nodes = self.estructura.keys()
        card = {}
        for value in nodes:
            card[value] = df[value].unique()
        return card
    
    def generate(self, df):
        factores = {}
        parents = self.parents
        for child, parent_list in parents.items():
            if (len(parent_list) == 0):
                factores[tuple(child)] = estimar_marginal(df,child, self.alpha)
            else:
                key = parent_list.copy()
                key.insert(0, child)
                key = tuple(key)
                factores[key] = estimar_condicional(df,child, parent_list, self.alpha)
        return factores
    
    def printFactores(self):
        for key, df in self.factores.items():
            print("**************")
            print (key)
            print(df)
        
    
    def get_scope(self, variable):
        # lista de factores
        factores = []
        for factor in self.factores.keys():
            if(variable in factor):
                factores.append(factor)
        return factores
    

    def get_hidden_variables(self, consulta, evidencia):
        nodes = self.estructura.keys()
        parents = self.parents
        hidden_variables = []

        for variable in nodes:
            if ((variable not in evidencia) and variable != consulta):
                hidden_variables.append([variable, len(parents[variable])])
        
      
        
        sorted(hidden_variables, key=lambda item: item[1])
        
        Z = []
        for i in hidden_variables:
            Z.append(i[0])
        return Z
    
    def estimar_marginal(self, factor, valores):
     
        assert len(valores) == 1, "Marginal solo acepta un valor"
        resultado = self.factores[factor]
        resultado = resultado[resultado[factor[0]]==valores[0]]
        return list(resultado['P'])[0]
    
    def estimar_condicional(self, factor, valores):
        resultado = self.factores[factor]
        for i in range(len(factor)):
            index = factor[i]
            value = valores[i]
            #filtrar la columna 
            resultado = resultado[resultado[index] == value]
        return list(resultado['P'])[0]

        
        
def inferencia(clase, e_variables , e_values ,df, red_bayesiana, alpha):
    
    # [Y,z] , [0, 1]
    
    var_val = {}
    
    for i in range(len(e_variables)):
        var_val[e_variables[i]] = e_values[i]
    
    
    # retorna un diccionario con llaves los nodos y valores 
    parentsList = get_parents(red_bayesiana)
    
   
    
    card = cardinalidad(df, clase)
    
    
    
    
    list_prob = []
    
    for valor in card:
        p = 1
        var_val[clase] = valor
        for var, parents in parentsList.items():
            if (len(parents) == 0):
                p = p * estimar_marginal_valor(df, var, var_val[var] , alpha )
             
            else:
                
                parents_values = []
                
                for parent in parentsList[var]:
                    parents_values.append(var_val[parent])
                
                cond = estimar_condicional_valor(df,var, parentsList[var], parents_values, alpha )
              
                
                cond = cond[cond[var] == var_val[var]]
                
                cond_val = cond.iloc[0]['P']
                
                p = p * cond_val
      
        list_prob.append(p)
    
    columns = {}
    
    columns[clase] = card
    columns['P'] = list_prob
    
    return pd.DataFrame(columns)


df_train = leer_dataset("training_dataset1.csv")


estructura = {'A':['C'],'C': ['B'], 'B':[]}


red = Red_Bayesiana(df_train,estructura, 1.0)
red.printFactores()

inferencia('C',['A','B'],[2,1], df_train, estructura,1.0)

0.375
**************
('A',)
   A         P
0  0  0.384615
1  2  0.307692
2  1  0.307692
**************
('C', 'A')
   C  A         P
0  0  0  0.666667
1  1  0  0.333333
2  0  2  0.600000
3  1  2  0.400000
4  0  1  0.200000
5  1  1  0.800000
**************
('B', 'C')
   B  C         P
0  0  0  0.428571
1  1  0  0.571429
2  0  1  0.285714
3  1  1  0.714286


,C,P
0,0,0.105495
1,1,0.087912


# Inferencia con variables ocultas

## Implementación

In [6]:


def get_scope(factores, variable):
    factores_with_variable = []
    factores_without_variable = []
    for factor in factores:
        if(variable in factor):
            factores_with_variable.append(factor)
        else:
            factores_without_variable.append(factor)
    return (factores_with_variable, factores_without_variable)

def get_vars(phi_prima):
    involved = []
  
    for element in phi_prima:
        for value in element:
            if(value not in involved):
                involved.append(value)
  
    return involved

def sum_product_eliminate_var(factores, variable_to_eliminate, consulta, valor_consulta, evidencia, valor_evidencia , red_bayesiana ):
    # factores que se van a factorizar
    
    phi_prima, phi_prima_prima = get_scope(factores,variable_to_eliminate)
      
  
    # lista de variables involucradas en phi_prima
    variables_in_scope = get_vars(phi_prima)

    #diccionario: llave -> variable , valor -> lista de cardinalidad
    
    var_to_values = {}
    
    for var in variables_in_scope:
        if (var != consulta and (var not in evidencia)):
            var_to_values[var] = red_bayesiana.card[var]
    var_to_values[consulta] = [valor_consulta]
    
    for i in range(len(evidencia)):
        var_to_values[evidencia[i]] = [valor_evidencia]

 
    
    # llave: variable , valor --> indice en una n_upla
    var_to_index = {}

    #total number of posibilites
    n =  1
    for var, val in var_to_values.items():
        n = n*len(val)
  
    posibilidades =  []
    for i in range(n):
        posibilidades.append([])
    #
    groups = 1
    
    
    pos = 0
    for var in variables_in_scope:
        var_to_index[var] = pos
        pos = pos + 1
        values = var_to_values[var]
        m = len(values)
        groups = groups*m
        rep = int(n/groups)
        for i in range(groups):
            for j in range(rep):
                posibilidades[i*rep + j].extend([values[i%m]])
  # factores que quedan
    
    p = 0 
  # iterar entre todas las posibilidades
    for n_upla in posibilidades:
        p_local = 1
        for dist  in phi_prima:
            var = dist[0]
            cond = dist[1:]
            
            

            # hallar los valores que tomas las variables en la distribuiciones
            if (len(cond) == 0):
                p_marginal = red_bayesiana.estimar_marginal(dist,[n_upla[var_to_index[var]]])
                p_local = p_local*p_marginal
            else:
                valores = []
                for var in dist:
                    index = var_to_index[var]
                    valores.append(n_upla[index])
                
                
                p_condicional = red_bayesiana.estimar_condicional(dist, valores)
              
                p_local = p_local*p_condicional
        p = p + p_local
  # sumatoria de los valores en los factores dentro del get_scope

    return [phi_prima_prima, p]

def sum_product(factores, consulta, valor_consulta, evidencia, valor_evidencia, red_bayesiana):
    
    var_to_val = {}
    for i in range(len(evidencia)):
        var_to_val[evidencia[i]] = valor_evidencia[i]
    
    var_to_val[consulta] = valor_consulta
    
    p = 1
    
    for dist in factores:
        var = dist[0]
        cond = dist[1:]
        if (len(cond) == 0):
            p_marginal = red_bayesiana.estimar_marginal(dist,[var_to_val[var]])
            p = p*p_marginal
        else:
            valores = []
            for var in dist:
                valores.append(var_to_val[var])
         

            p_condicional = red_bayesiana.estimar_condicional(dist, valores)
           
            p = p*p_condicional
    return p

def sum_product_ve(consulta, valor_consulta , evidencia, valor_evidencia, red_bayesiana):
    Z = red_bayesiana.get_hidden_variables(consulta, evidencia)
    
    tetha = []
    factores = red_bayesiana.factores.keys() # lista de factores
    for i in range(len(Z)):
  
        
        if(len(factores) == 0):
            break
        [nuevos_factores,sumatoria] =  sum_product_eliminate_var(factores, Z[i], consulta, 
                                                                 valor_consulta, evidencia, 
                                                                 valor_evidencia, red_bayesiana)
    
        factores = nuevos_factores
        tetha.append(sumatoria)
    
    if(len(factores)>0):
        p_last = sum_product(factores,consulta,  valor_consulta, evidencia,valor_evidencia, red_bayesiana)
        tetha.append(p_last)
    p = 1
    for sumatoria in tetha:
        p = p*sumatoria
    
    return p


def inferencia_con_variables_ocultas(consulta, evidencia, valor_evidencia, red_bayesiana):
    
    card = red_bayesiana.card[consulta]
    list_prob = []
    for valor_consulta in card:
        p = sum_product_ve(consulta, valor_consulta , evidencia, valor_evidencia, red_bayesiana)
        list_prob.append(p)
    columns = {}
    columns[consulta] = card
    columns['P'] = list_prob
    
    return pd.DataFrame(columns)



inferencia_con_variables_ocultas('C',[],[],red)


,C,P
0,0,0.502564
1,1,0.497436
